>原文地址 https://blog.csdn.net/qq_33733970/article/details/77719427?utm_medium=distribute.pc_relevant.none-task-blog-baidujs-1

GIL 与互斥锁再理解
===========

![](https://img-blog.csdn.net/20170830151042308?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcXFfMzM3MzM5NzA=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

线程一要把 python 代码交给解释器去执行，而此时垃圾回收线程和线程二也需要将自己的任务交给 python 解释器去执行，为了防止各个线程之间的数据产生冲突，谁拿到 GIL 锁的权限谁才能执行自己的任务，这就避免了不同任务之间的数据不会产生冲突，这是在同一个进程中加 GIL 锁会保证数据的安全，不同的数据要加不同的锁。

死锁与递归锁
======

死锁
--

### 代码演示

```py
from threading import Thread, Lock
import time
mutexA = Lock()
mutexB = Lock()

class MyThread(Thread):
    def run(self):
        self.f1()
        self.f2()

    def f1(self):
        mutexA.acquire()
        print('%s 拿到A锁' % self.name)
        mutexB.acquire()
        print('%s 拿到B锁' % self.name)
        mutexB.release()
        mutexA.release()

    def f2(self):
        mutexB.acquire()
        time.sleep(1)
        print('%s 拿到B锁' % self.name)
        mutexA.acquire()
        print('%s 拿到A锁' % self.name)
        mutexA.release()
        mutexB.release()


if __name__ == '__main__':
    for i in range(10):
        t = MyThread()
        t.start()
```

该种情况出现死锁：  
![](https://img-blog.csdn.net/20170830153434695?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcXFfMzM3MzM5NzA=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

### 代码讲解

由于 Thread-1 创建的比较快，所以 Thread-1 先抢到 A 锁，继而顺利成章的拿到 B 锁，当 Thread-1 释放掉 A 锁时，另外 9 个线程抢 A 锁，于此同时，Thread-1 抢到 B 锁，而此时 Thread-2 抢到 A 锁，这样 Thread-1、Thread-2 就等待彼此把锁释放掉，这样程序就卡住了，解决这个问题就用到了递归锁。

递归锁
---

### 代码演示

```py
from threading import Thread, Lock, RLock
import time

# mutexA = Lock()
# mutexB = Lock()
mutexA = mutexB = RLock()


class MyThread(Thread):
    def run(self):
        self.f1()
        self.f2()

    def f1(self):
        mutexA.acquire()
        print('%s 拿到A锁' % self.name)
        mutexB.acquire()
        print('%s 拿到B锁' % self.name)
        mutexB.release()
        mutexA.release()

    def f2(self):
        mutexB.acquire()
        time.sleep(1)
        print('%s 拿到B锁' % self.name)
        mutexA.acquire()
        print('%s 拿到A锁' % self.name)
        mutexA.release()
        mutexB.release()


if __name__ == '__main__':
    for i in range(10):
        t = MyThread()
        t.start()
```

### 代码讲解

递归锁时通过计数完成对锁的控制的，当 acquire 一次，count+=1，release 一次，count-=1，当 count=0，所有的线程都可以对锁进行抢夺。从而避免了死锁的产生。

信号量 Semaphore
=============

代码演示
----

```py
from threading import Thread, Semaphore, currentThread
import time
smph = Semaphore(5)


def do_task():
    smph.acquire()
    print('\033[45m%s\033[0m 获得了权限' % currentThread().name)
    time.sleep(2)
    print('\033[46m%s\033[0m 放弃了权限' % currentThread().name)
    smph.release()


if __name__ == '__main__':
    for i in range(10)
        t = Thread(target=do_task, )
        t.start() 
```

代码效果
----

![](https://img-blog.csdn.net/20170830155911339?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcXFfMzM3MzM5NzA=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast)

代码讲解
----

信号量 Semaphore 本质也是一把锁，但是这把锁可以限定允许多个任务同时执行任务，但是不能超出规定的限制，下面的代码参数 5 就代表可以执行 5 个任务，如果第 6 个任务要执行，必须等 5 个任务中的一个结束，然后第六个才能进入执行。

```
smph = Semaphore(5)
```

这有点像进程池，只不过进程池规定了进程数量，多个任务进入进程池只能有数量一定的进程进行处理。，但是 Semaphore 可以产生多个线程。

线程 Queue
========

队列 Queue
--------

### 代码演示

In [7]:
import queue

q = queue.Queue()
q.put('1')
q.put(1)
q.put({'a': 1})

print(q.get())
print(q.get())
print(q.get())

1
1
{'a': 1}


### 代码讲解

1.  先进先出
2.  可以存放任意类型数据

堆栈 Queue
--------

### 代码演示

In [6]:
import queue
q = queue.LifoQueue()
q.put(1)
q.put('1')
q.put({'a': 1})

print(q.get())
print(q.get())
print(q.get())


{'a': 1}
1
1


### 代码讲解

1.  可以存放任意数据类型
2.  Lifo 代表后进先出

优先级 Queue
---------

### 代码演示

In [5]:
import queue

q = queue.PriorityQueue()
q.put((10, 'Q'))
q.put((30, 'Z'))
q.put((20, 'A'))

print(q.get())
print(q.get())
print(q.get())

(10, 'Q')
(20, 'A')
(30, 'Z')


### 代码讲解

1.  存放的数据是元组类型，带有优先级数字越小优先级越高。
2.  数据优先级高的优先被取出。
3.  用于 VIP 用户数据优先被取出场景，因为上面两种都要挨个取出。

Event
=====

代码演示
----

```py
from threading import Thread, Event, currentThread
import time

e = Event()


def traffic_lights():
    time.sleep(5)
    e.set()


def cars():
    print('\033[45m%s\033[0m is waiting' % currentThread().name)
    e.wait()
    print('\033[45m%s\033[0m is running' % currentThread().name)


if __name__ == '__main__':
    for i in range(10):
        t = Thread(target=cars, )
        t.start()
    traffic_lights = Thread(target=traffic_lights, )
    traffic_lights.start()
```

### 代码讲解

首先创建 10 个线程代表 10 辆车正在等信号灯，创建 1 个线程代表信号灯，当 10 辆汽车被创建后就等着信号灯发信号起跑，当遇到 e.wait() 时程序被挂起，等待信号灯变绿，而 e.set() 就是来改变这个状态让信号灯变绿，当 e.set 被设置后 cars 等到了信号，就可以继续往后跑了，代码可以继续执行了。e.set() 默认 False，e.set() 调用后值变为 True，e.wait() 接收到后程序由挂起变为可执行。

### 应用场景

#### 代码演示

```py
from threading import Thread, Event, currentThread
import time

e = Event()


def check_sql():
    print('%s is checking mySQL' % currentThread().name)
    time.sleep(5)
    e.set()


def link_sql():
    count = 1
    while not e.is_set():#e.isSet是一个绑定方法，自带布尔值为True,e.is_set()默认值为False
        e.wait(timeout=1)
        print('%s is trying %s' % (currentThread().name, count))
        if count > 3:
            raise ConnectionError('连接超时')
        count += 1
    print('%s is connecting' % currentThread().name)


if __name__ == '__main__':
    t_check = Thread(target=check_sql, )
    t_check.start()
    for i in range(3):
        t_link = Thread(target=link_sql, )
        t_link.start()
```

#### 代码讲解

1.  数据库远程连接
2.  e.isSet 是一个绑定方法，自带布尔值为 True，e.is_set() 默认值为 False

定时器
===

代码演示
----

In [4]:
from threading import Timer
import time


def deal_task(n):
    print('%s 我被执行了~' % n)


print(int(time.time()))
t = Timer(3, deal_task, args=(10,))
t.start()
t.join()
print(int(time.time()))

1590843636
10 我被执行了~
1590843639


代码讲解
----

注意传参时必须是元组形式